In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
import sys
sys.path.insert(0, '/media/mistertandon/DATA/git_repos/ai/01-llm-rsbt/ch_02')

from ch02_26_data_sampling import GPTDatasetV1, create_dataloader_v1

In [4]:
import sys
sys.path.insert(0, '/media/mistertandon/DATA/git_repos/ai/01-llm-rsbt/ch_03')

from ch_03_3_1_simple_self_attention import naive_softmax

In [5]:
vocab_size = 50257
out_dim = 3
token_embedding_layer = torch.nn.Embedding(num_embeddings=vocab_size, embedding_dim=out_dim)
print(token_embedding_layer)

Embedding(50257, 3)


In [6]:
raw_text = "millionaire journey start with first step"

In [7]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=6, stride=6, shuffle=False, drop_last=False, num_workers=0)
data_iter = iter(dataloader)
print(data_iter)

In [8]:
inputs, targets = next(data_iter)

In [9]:
print(inputs)

tensor([[14100,  7626,  7002,   923,   351,   717]])


In [10]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([1, 6, 3])


In [11]:
token_embeddings[0, 0]

tensor([-0.2350,  0.4348, -1.2125], grad_fn=<SelectBackward0>)

In [12]:
position_embedding_layer = torch.nn.Embedding(6, 3)
print(position_embedding_layer)


Embedding(6, 3)


In [13]:
positional_embeddings = position_embedding_layer(torch.arange(6))
print(positional_embeddings.shape)

torch.Size([6, 3])


In [14]:
_inputs_embeddings = token_embeddings + positional_embeddings
inputs_embeddings = _inputs_embeddings.squeeze(0)
print(inputs_embeddings.shape)
print(inputs_embeddings)

torch.Size([6, 3])
tensor([[-0.6607, -1.1075, -1.3587],
        [-0.6977,  0.0217, -0.2162],
        [-0.7956, -0.7602, -0.2818],
        [ 0.0614,  0.6362, -0.0563],
        [ 1.5385,  0.7415,  1.2035],
        [-0.0463,  0.2891,  0.5057]], grad_fn=<SqueezeBackward1>)


No we'll try to compute attention score corresponding to q^2 i.e. journey

attention_score_2 is un-normalized attention score represneted by omega for 2nd element (At index 1)

In [ ]:
"""
Uncomment the code below to replace the previous inputs_embeddings with the following implementation.
Here, we explicitly compute the context vector for the second element (“journey”) in the sequence using a for-loop–based approach.
Subsequently, this computation can be generalized to derive context vectors for all elements in the sequence through matrix-based operations.
inputs_embeddings = torch.tensor(
[[0.43, 0.15, 0.89], # Your
[0.55, 0.87, 0.66], # journey
[0.57, 0.85, 0.64], # starts
[0.22, 0.58, 0.33], # with
[0.77, 0.25, 0.10], # one
[0.05, 0.80, 0.55]] # step
)
"""

In [16]:
query = inputs_embeddings[1]
attention_score_2 = torch.empty(inputs_embeddings.shape[0])
for idx, inputs_embedding in enumerate(inputs_embeddings):
    attention_score_2[idx] = torch.dot(query, inputs_embedding)

print(attention_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


attention_weight_2_normalized is normalized attention score (Known as attention weigth Represented by Alpha) for 2nd element (At index 1)

In [17]:
attention_weight_2_normalized = torch.softmax(attention_score_2, dim=0)
print(attention_weight_2_normalized)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])


In [18]:
attention_weight_2_naive_normalized =naive_softmax(attention_score_2)
print(attention_weight_2_naive_normalized)
print(attention_weight_2_naive_normalized.sum())

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [19]:
context_vec_2 = torch.zeros(query.shape)
for idx, input_embedding in enumerate(inputs_embeddings):
    context_vec_2 += attention_weight_2_naive_normalized[idx] * input_embedding
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])
